In [1]:
import numpy as np
import pandas as pd

from beakerx.object import beakerx
from sklearn.model_selection import train_test_split

import cimcb_lite as cb

beakerx.pandas_display_table()  # by default display pandas tables as BeakerX interactive tables

print('All packages successfully loaded')

BeakerxHTML(value='You need beakerx_tabledisplay to use this')

All packages successfully loaded


In [3]:
# The path to the input file (Excel spreadsheet)
filename = '/Users/davidchen/Documents/GitHub/Metabolomics-Workflow/GastricCancer_NMR.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 140 TOTAL PEAKS: 149
Done!


In [4]:
display(dataTable) # View and check the dataTable 

,Idx,SampleID,SampleType,Class,M1,M2,M3,M4,M5,M6,...,M140,M141,M142,M143,M144,M145,M146,M147,M148,M149
1,1,sample_1,QC,QC,90.1,491.6,202.9,35.0,164.2,19.7,...,115.1,64.8,25.5,473.9,26.5,NaN,6.8,118.6,710.6,203.6
2,2,sample_2,Sample,GC,43.0,525.7,130.2,NaN,694.5,114.5,...,84.2,357.1,16.1,455.5,29.5,28.1,35.8,316.1,390.7,199.0
3,3,sample_3,Sample,BN,214.3,10703.2,104.7,46.8,483.4,152.3,...,993.5,1698.5,32.9,75.9,33.2,802.8,967.6,154.4,31.6,195.2
4,4,sample_4,Sample,HE,31.6,59.7,86.4,14.0,88.6,10.3,...,58.1,83.5,60.5,136.9,17.0,10.2,24.7,64.1,91.4,91.6
5,5,sample_5,Sample,GC,81.9,258.7,315.1,8.7,243.2,18.4,...,44.5,47.6,45.6,1441.7,35.2,0.1,22.8,135.0,322.3,254.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,sample_136,QC,QC,97.6,341.1,232.1,38.1,174.0,7.7,...,79.7,101.8,23.9,296.0,25.0,NaN,7.5,141.5,675.7,200.8
137,137,sample_137,Sample,GC,405.3,510.7,521.9,91.9,732.1,145.7,...,434.8,84.8,182.3,110.7,123.9,0.4,36.3,60.1,317.3,401.7
138,138,sample_138,Sample,BN,45.4,191.6,41.0,18.7,40.8,32.2,...,45.3,44.5,14.5,83.8,27.9,0.3,0.5,47.3,47.8,46.5
139,139,sample_139,Sample,HE,30.7,56.8,35.9,20.9,17.4,NaN,...,28.0,38.7,1.3,130.9,24.6,0.7,5.9,20.7,124.1,28.9


In [5]:
display(peakTable) # View and check PeakTable

,Idx,Name,Label,Perc_missing,QC_RSD
1,1,M1,1_3-Dimethylurate,11.428571,32.208005
2,2,M2,1_6-Anhydro-β-D-glucose,0.714286,31.178028
3,3,M3,1_7-Dimethylxanthine,5.000000,34.990605
4,4,M4,1-Methylnicotinamide,8.571429,12.804201
5,5,M5,2-Aminoadipate,1.428571,9.372664
...,...,...,...,...,...
145,145,M145,uarm1,23.571429,41.406985
146,146,M146,uarm2,4.285714,34.458172
147,147,M147,β-Alanine,1.428571,27.623517
148,148,M148,π-Methylhistidine,1.428571,16.561921


In [6]:

rsd = peakTable['QC_RSD']  
percMiss = peakTable['Perc_missing']  
peakTableClean = peakTable[(rsd < 20) & (percMiss < 10)]   

print("Number of peaks remaining: {}".format(len(peakTableClean)))

Number of peaks remaining: 52


In [9]:

peaklist = peakTableClean['Name']                   # Set peaklist to the metabolite names in the peakTableClean
X = dataTable[peaklist].values                      # Extract X matrix from dataTable using peaklist
Xlog = np.log10(X)                                  # Log scale (base-10)
Xscale = cb.utils.scale(Xlog, method='auto')        # methods include auto, pareto, vast, and level
Xknn = cb.utils.knnimpute(Xscale, k=3)              # missing value imputation (knn - 3 nearest neighbors)

print("Xknn: {} rows & {} columns".format(*Xknn.shape))

cb.plot.pca(Xknn,
            pcx=1,                                                  # pc for x-axis
            pcy=2,                                                  # pc for y-axis
            group_label=dataTable['SampleType'])                    # labels for Hover in PCA loadings plot


Xknn: 140 rows & 52 columns


Loading BokehJS ...

In [10]:
# Select subset of Data for statistical comparison
dataTable2 = dataTable[(dataTable.Class == "GC") | (dataTable.Class == "HE")]  # Reduce data table only to GC and HE class members
pos_outcome = "GC" 

# Calculate basic statistics and create a statistics table.
statsTable = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean,
                                        group='Class',                # Column used to determine the groups
                                        posclass=pos_outcome,         # Value of posclass in the group column
                                        parametric=True)              # Set parametric = True or False

# View and check StatsTable
display(statsTable)

,Idx,Name,Label,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
1,4,M4,1-Methylnicotinamide,51.739474,"(39.35, 64.13)",26.477778,"(19.98, 32.98)",0,3.482846,0.000848,0.008816,9,10.843,5.0,16.279,0.861608,9.126937e-07,9.835944,0.002478
2,5,M5,2-Aminoadipate,169.915000,"(115.14, 224.69)",265.118605,"(146.65, 383.59)",1,-1.395129,0.166791,0.420837,0,0.000,0.0,0.000,0.551547,1.591575e-14,1.714528,0.194101
3,7,M7,2-Furoylglycine,53.987179,"(31.17, 76.81)",118.525581,"(78.5, 158.55)",1,-2.672784,0.009114,0.059243,1,1.205,2.5,0.000,0.696827,1.009690e-11,5.909098,0.017299
4,8,M8,2-Hydroxyisobutyrate,79.267500,"(59.69, 98.85)",54.395349,"(42.53, 66.26)",0,2.163519,0.033448,0.158119,0,0.000,0.0,0.000,0.817766,9.915664e-09,3.216053,0.076652
5,11,M11,3-Aminoisobutyrate,171.279487,"(104.01, 238.55)",201.343902,"(107.59, 295.1)",0,-0.506244,0.614113,0.760330,3,3.614,2.5,4.651,0.629707,6.749377e-13,0.241249,0.624685
6,14,M14,3-Hydroxyisobutyrate,83.902500,"(58.8, 109.01)",61.531707,"(45.75, 77.31)",0,1.486528,0.141120,0.407680,2,2.410,0.0,4.651,0.730224,6.725048e-11,1.852628,0.177348
7,15,M15,3-Hydroxyisovalerate,62.300000,"(48.06, 76.54)",58.472093,"(44.97, 71.98)",0,0.382551,0.703054,0.812418,0,0.000,0.0,0.000,0.820793,1.225706e-08,0.110564,0.740362
8,25,M25,6-Hydroxynicotinate,20.640000,"(15.86, 25.42)",30.293023,"(21.28, 39.31)",1,-1.814600,0.073288,0.238185,0,0.000,0.0,0.000,0.735144,6.198545e-11,1.828066,0.180119
9,26,M26,ATP,22.813514,"(17.62, 28.0)",39.944737,"(20.29, 59.6)",1,-1.632723,0.106834,0.326785,8,9.639,7.5,11.628,0.455527,3.358144e-15,2.474440,0.120035
10,31,M31,Adipate,57.615000,"(31.6, 83.63)",80.041860,"(18.98, 141.1)",0,-0.645264,0.520580,0.694410,0,0.000,0.0,0.000,0.359728,2.820873e-17,0.466944,0.496346


In [11]:
# Save StatsTable to Excel
statsTable.to_excel("stats.xlsx", sheet_name='StatsTable', index=False)
print("done!")

done!


In [12]:
# Create a Binary Y vector for stratifiying the samples
outcomes = dataTable2['Class']                                  # Column that corresponds to Y class (should be 2 groups)
Y = [1 if outcome == 'GC' else 0 for outcome in outcomes]       # Change Y into binary (GC = 1, HE = 0)  
Y = np.array(Y)                                                 # convert boolean list into to a numpy array

In [13]:
dataTrain, dataTest, Ytrain, Ytest = train_test_split(dataTable2, Y, test_size=0.25, stratify=Y,random_state=10)

print("DataTrain = {} samples with {} positive cases.".format(len(Ytrain),sum(Ytrain)))
print("DataTest = {} samples with {} positive cases.".format(len(Ytest),sum(Ytest)))

DataTrain = 62 samples with 32 positive cases.
DataTest = 21 samples with 11 positive cases.


In [14]:
peaklist = peakTableClean['Name']                           # Set peaklist to the metabolite names in the peakTableClean
XT = dataTrain[peaklist]                                    # Extract X matrix from DataTrain using peaklist
XTlog = np.log(XT)                                          # Log scale (base-10)
XTscale = cb.utils.scale(XTlog, method='auto')              # methods include auto, pareto, vast, and level
XTknn = cb.utils.knnimpute(XTscale, k=3)                    # missing value imputation (knn - 3 nearest neighbors)

In [18]:
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals

# run the cross validation
cv.run()  

Kfold: 100%|██████████| 100/100 [00:02<00:00, 36.36it/s]


In [19]:
cv.plot() # plot cross validation statistics

Loading BokehJS ...

In [20]:
modelPLS = cb.model.PLS_SIMPLS(n_components=2)  # Initalise the model with n_components = 2

In [21]:
Ypred = modelPLS.train(XTknn, Ytrain) 

In [22]:
# Evaluate the model 
modelPLS.evaluate(cutoffscore=0.5)  

Loading BokehJS ...

In [23]:
modelPLS.permutation_test(nperm=100) #nperm refers to the number of permutations

Permutation Resample: 100%|██████████| 100/100 [00:00<00:00, 128.94it/s]


Loading BokehJS ...

In [24]:
modelPLS.plot_projections(label=dataTrain[['Idx','SampleID']], size=12) # size changes circle size

Loading BokehJS ...

In [25]:
# Calculate the bootstrapped confidence intervals 
modelPLS.calc_bootci(type='bca', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet = modelPLS.plot_featureimportance(peakTableClean,
                                            peaklist,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

Jackknife Resample: 100%|██████████| 62/62 [00:00<00:00, 1636.40it/s]


Loading BokehJS ...

In [26]:
# Get mu and sigma from the training dataset to use for the Xtest scaling
mu, sigma  = cb.utils.scale(XTlog, return_mu_sigma=True) 

In [27]:
# Pull of Xtest from DataTest using peaklist ('Name' column in PeakTable)
peaklist = peakTableClean.Name 
XV = dataTest[peaklist].values

# Log transform, unit-scale and knn-impute missing values for Xtest
XVlog = np.log(XV)
XVscale  = cb.utils.scale(XVlog, method='auto', mu=mu, sigma=sigma) 
XVknn = cb.utils.knnimpute(XVscale, k=3)

In [28]:
# Calculate Ypredicted score using modelPLS.test
YVpred = modelPLS.test(XVknn)

# Evaluate Ypred against Ytest
evals = [Ytest, YVpred]    # alternative formats: (Ytest, Ypred) or np.array([Ytest, Ypred])
#modelPLS.evaluate(evals, specificity=0.9)
modelPLS.evaluate(evals, cutoffscore=0.5) 

Loading BokehJS ...

In [29]:
dataSheet = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet['Ypred'] = YVpred 
 
display(dataSheet) # View and check the dataTable 

,Idx,SampleID,Class,Ypred
4,4,sample_4,HE,0.596320
78,78,sample_78,GC,0.762340
90,90,sample_90,HE,0.190719
71,71,sample_71,GC,0.799896
92,92,sample_92,GC,1.046995
119,119,sample_119,HE,0.122116
56,56,sample_56,HE,0.166406
104,104,sample_104,HE,-0.130498
98,98,sample_98,GC,0.230983
36,36,sample_36,GC,0.401337


In [30]:
# Create an empty excel workbook
writer = pd.ExcelWriter("modelPLS.xlsx")     # provide the filename for the Excel file

# Add each dataframe to the workbook in turn, as a separate worksheet
dataSheet.to_excel(writer, sheet_name='Datasheet', index=False)
peakSheet.to_excel(writer, sheet_name='Peaksheet', index=False)

# Write the Excel workbook to disk
writer.save()

print("Done!")

Done!


In [31]:
pwd


'/Users/davidchen/Documents/GitHub/Erevna'